In [1]:
!pip install pytesseract evaluate tqdm transformers datasets rouge-score accelerate nltk tensorboard jupyter-black py7zr --upgrade
!pip install bitsandbytes
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 40.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.0 MB/

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

#현재 폴더 경로; 작업 폴더 기준
print(os.getcwd())

/content


In [4]:
os.chdir("/content/drive/MyDrive/chart2text/FLANT5")

In [5]:
print(os.getcwd())

/content/drive/MyDrive/chart2text/FLANT5


In [6]:
import os
import json
def load_dataset():


    pew_dataset_root_path='../dataset/pew_dataset_reduced'
    statista_dataset_root_path='../dataset/statista_dataset_reduced'
    scicap_data_root_path='../dataset/scicap_data_reduced'
    simulated_scatter_root_path='../dataset/simulated_scatter_dataset'

    # pew_dataset_root_path='/Users/dongunyun/study/datascience/chart2text/dataset/pew_dataset'
    # statista_dataset_root_path='/Users/dongunyun/study/datascience/chart2text/dataset/statista_dataset'
    # scicap_data_root_path='/Users/dongunyun/study/datascience/chart2text/dataset/scicap_data'
    # simulated_scatter_root_path='/Users/dongunyun/study/datascience/chart2text/dataset/simulated_scatter_dataset'

    train_dataset=[]
    valid_dataset=[]
    test_dataset=[]

    capspath=os.path.join(simulated_scatter_root_path,'data','train')
    imagepath=os.path.join(simulated_scatter_root_path,'image','train')
    fileEx = r'.json'
    file_list = [file.split('.')[0] for file in os.listdir(capspath) if file.endswith(fileEx)]

    for filename in file_list:
        image_path=os.path.join(imagepath,f'{filename}.png')
        cap_path=os.path.join(capspath,f'{filename}.json')
        with open(cap_path) as f:
            json_object = json.load(f)
        if "description_rewrite" in json_object:
            train_dataset.append({'image':image_path,'text':json_object['description_rewrite'],'origin_text':cap_path})

    capspath=os.path.join(simulated_scatter_root_path,'data','valid')
    imagepath=os.path.join(simulated_scatter_root_path,'image','valid')
    fileEx = r'.json'
    file_list = [file.split('.')[0] for file in os.listdir(capspath) if file.endswith(fileEx)]

    for filename in file_list:
        image_path=os.path.join(imagepath,f'{filename}.png')
        cap_path=os.path.join(capspath,f'{filename}.json')
        with open(cap_path) as f:
            json_object = json.load(f)
        if "description_rewrite" in json_object:
            valid_dataset.append({'image':image_path,'text':json_object['description_rewrite'],'origin_text':cap_path})

    capspath=os.path.join(simulated_scatter_root_path,'data','test')
    imagepath=os.path.join(simulated_scatter_root_path,'image','test')
    fileEx = r'.json'
    file_list = [file.split('.')[0] for file in os.listdir(capspath) if file.endswith(fileEx)]

    for filename in file_list:
        image_path=os.path.join(imagepath,f'{filename}.png')
        cap_path=os.path.join(capspath,f'{filename}.json')
        with open(cap_path) as f:
            json_object = json.load(f)
        if "description_rewrite" in json_object:
            test_dataset.append({'image':image_path,'text':json_object['description_rewrite'],'origin_text':cap_path})

    dataset=train_dataset
    # dataset['valid']=valid_dataset
    # dataset['test']=test_dataset
    dataset.extend(test_dataset)
    return dataset

In [7]:
dataset = load_dataset()
print(len(dataset))

3704


In [8]:
import pandas as pd
from datasets import Dataset
df=pd.DataFrame(dataset,columns=['image','text','origin_text'])
data = Dataset.from_pandas(df).train_test_split(test_size=0.2, seed=0)
print(data)

DatasetDict({
    train: Dataset({
        features: ['image', 'text', 'origin_text'],
        num_rows: 2963
    })
    test: Dataset({
        features: ['image', 'text', 'origin_text'],
        num_rows: 741
    })
})


In [9]:
from datasets import concatenate_datasets
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
from transformers import DataCollatorForSeq2Seq
from huggingface_hub import HfFolder
from transformers import pipeline
from random import randrange
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import os


In [10]:
MODEL_NAME = "chart2text"  # the name of your model
MODEL_ID = "/content/drive/MyDrive/chart2text/FLANT5/output/summary_chart-checkpoint-last" # the id of the base model we will train (can be small, base, large, xl, etc.) (the bigger - the more GPU memory you need)
# REPOSITORY_ID = f"{MODEL_ID.split('/')[1]}-{MODEL_NAME}"  # the id of your huggingface repository where the model will be stored
REPOSITORY_ID="./output"

NUM_TRAIN_EPOCHS = 4  # number of epochs to train
max_source_length=2048+512
max_target_length=512
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base",device_map="auto")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:
def preprocess_function(sample, padding="max_length"):

    def getinputs(cap_path):
        if os.path.isfile(cap_path):
            with open(cap_path) as f:
                json_object = json.load(f)
            if 'description' in json_object:
                if "This scatterplot is a chart showing the distribution of values over time" in json_object["description"]:
                    query="describe this data distribution about trend and its abnormalities"
                elif "This scatterplot is a graph that represents the relationship between" in json_object["description"]:
                    query="describe this data distribution about correlation and its abnormalities"
                elif "This scatterplot shows the clustering results of groups between" in json_object["description"]:
                    query="describe this data distribution about differences and clustering and its abnormalities"
                else:
                    query="describe this data distribution"
            else:
                query="describe this data distribution"
        else:
            query="describe this data distribution"


        inputs = str(json_object['features']) + "Look at the data table above, " + query + "\n\n"
        return inputs

    inputs = [getinputs(input) for input in sample['origin_text']]
    model_inputs = tokenizer(
        inputs, max_length=max_source_length, padding=padding, truncation=True
    )
    labels = tokenizer(
        text_target=sample["text"],
        max_length=max_target_length,
        padding=padding,
        truncation=True,
    )

    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label]
            for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_dataset = data.map(
    preprocess_function, batched=True, remove_columns=["image"]
)

Map:   0%|          | 0/2963 [00:00<?, ? examples/s]

Map:   0%|          | 0/741 [00:00<?, ? examples/s]

In [12]:
# metric
metric = evaluate.load("rouge")
nltk.download("punkt")


# postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels


# compute metrics
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:
label_pad_token_id = -100

data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, label_pad_token_id=label_pad_token_id, pad_to_multiple_of=8
)

In [16]:
# Define training args
training_args = Seq2SeqTrainingArguments(
    # training parameters
    output_dir=REPOSITORY_ID,
    overwrite_output_dir = 'True',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    fp16=False,  # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=20,
    # logging & evaluation strategies
    logging_dir=f"{REPOSITORY_ID}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",

    save_total_limit=1,

    report_to="tensorboard",

    save_strategy="epoch",
    load_best_model_at_end=True,

    disable_tqdm=False,
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.956000,0.806279,24.428800,17.948700,23.200300,23.980500,19.000000
2,0.813500,0.755012,24.344000,18.090600,23.034000,23.891700,19.000000
3,0.759300,0.728100,24.453100,18.686500,23.405400,24.068300,19.000000
4,0.723200,0.718561,24.362000,18.671100,23.370500,23.975100,19.000000
5,0.703500,0.713516,24.553300,18.908000,23.609100,24.181400,19.000000
6,0.674600,0.701941,24.595500,19.033800,23.682500,24.241200,19.000000
7,0.663600,0.702384,24.396700,18.766700,23.490300,24.032600,19.000000
8,0.644600,0.699252,24.279000,18.750900,23.385200,23.920900,19.000000
9,0.640700,0.703605,24.394300,18.851500,23.478900,24.036400,19.000000
10,0.617000,0.710302,24.509000,18.905900,23.583700,24.138300,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-635967aca760>", line 2, in <cell line: 2>
    trainer.train()
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1537, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1914, in _inner_training_loop
    self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_keys_for_eval)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2254, in _maybe_log_save_evaluate
    self.log(logs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2655, in log
    self.control = self.callback_handler.on_log(self.args, self.state, self.control, logs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer_callback.py"

In [ ]:
save_path = os.path.join('./output/', 'summary_chart-checkpoint-last')
model.save_pretrained(save_path, from_pt=True)
tokenizer.save_pretrained(save_path, from_pt=True)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-76ac3ac66ea2>", line 2, in <cell line: 2>
    model.save_pretrained(save_path, from_pt=True)
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 2225, in save_pretrained
    os.makedirs(save_directory, exist_ok=True)
  File "/usr/lib/python3.10/os.py", line 215, in makedirs
    makedirs(head, exist_ok=exist_ok)
  File "/usr/lib/python3.10/os.py", line 225, in makedirs
    mkdir(name, mode)
OSError: [Errno 107] Transport endpoint is not connected: './output'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' obje

In [13]:
MODEL_NAME = "chart2text"  # the name of your model
MODEL_ID ="/content/drive/MyDrive/chart2text/FLANT5/output/checkpoint-14820"
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained(MODEL_ID,device_map="auto")




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
idx=1

import torch
outputs = model.generate(input_ids=torch.tensor(tokenized_dataset["test"][idx]['input_ids']).unsqueeze(0).to('cuda'),max_new_tokens=1024)
decoded_preds = tokenizer.batch_decode(tokenized_dataset["test"][idx]['input_ids'], skip_special_tokens=True)
print(decoded_preds)
decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(decoded_preds)
tokenized_dataset["test"][idx]['text']

['', '<unk>', "'", 'x', '-', 'axi', 's', '-', 'l', 'abel', "'", ':', '', "'", 'date', 'time', "'", ',', '', "'", 'y', '-', 'axi', 's', '-', 'l', 'abel', "'", ':', '"', "'", 'nov', 'ice', "'", '",', '', "'", 'x', '-', 'axi', 's', '-', 'feature', 's', "'", ':', '[', '<unk>', "'", 'group', "'", ':', '', "'", 'y', 'r', "'", ',', '', "'", 'color', "'", ':', '', "'", '#', 'f', 'f', 'd', '500', "'", ',', '', "'", 'min', "'", ':', '', "'", '20', '23', '-11', '-22', '', '05', ':', '58', ':', '1', '0.000', "'", ',', '', "'", 'max', "'", ':', '', "'", '20', '23', '-11', '-25', '13', ':', '57', ':', '2', '2.000', "'", '<unk>', ',', '', '<unk>', "'", 'group', "'", ':', '', "'", 'B', 'K', "'", ',', '', "'", 'color', "'", ':', '', "'", '#', '00', 'c', '3', 'f', 'f', "'", ',', '', "'", 'min', "'", ':', '', "'", '20', '23', '-12', '-16', '08', ':', '48', ':', '1', '4.000', "'", ',', '', "'", 'max', "'", ':', '', "'", '20', '23', '-12', '-18', '08', ':', '21', ':', '26', '.', '000', "'", '<unk>', ',', '

'"This scatterplot displays the distribution of values over time for \'novice\' with a mean of -1245.172 and a standard deviation of 12.288. There are a total of 13 groups, and no other abnormalities are present."'

In [21]:
idx=2

import torch
outputs = model.generate(input_ids=torch.tensor(tokenized_dataset["test"][idx]['input_ids']).unsqueeze(0).to('cuda'),max_new_tokens=1024)
decoded_preds = tokenizer.batch_decode(tokenized_dataset["test"][idx]['input_ids'], skip_special_tokens=True)
print(decoded_preds)
decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(decoded_preds)
tokenized_dataset["test"][idx]['text']

['', '<unk>', "'", 'x', '-', 'axi', 's', '-', 'l', 'abel', "'", ':', '"', "'", 'woman', "'", '"', ',', '', "'", 'y', '-', 'axi', 's', '-', 'l', 'abel', "'", ':', '"', '', "'", 'vo', 'tes', "'", '",', '', "'", 'x', '-', 'axi', 's', '-', 'feature', 's', "'", ':', '[', '<unk>', "'", 'group', "'", ':', '', "'", 'LF', "'", ',', '', "'", 'color', "'", ':', '', "'", '#', 'b', 'd', '0000', "'", ',', '', "'", 'me', 'an', "'", ':', '14', '4.0', '56', '208', '79', '29', '70', '86', ',', '', "'", 's', 't', 'd', "'", ':', '11.', '85', '30', '83', '69', '180', '54', '64', ',', '', "'", 'media', 'n', "'", ':', '14', '5.2', '53', '94', '47', '48', '48', '204', ',', '', "'", 'min', "'", ':', '', '117', '.', '67', '222', '85', '47', '74', '117', ',', '', "'", 'max', "'", ':', '', '169', '.', '243', '520', '66', '65', '29', ',', '', "'", 'ske', 'w', 'ness', "'", ':', '', '-', '0.2', '34', '25', '365', '14', '05', '24', '28', ',', '', "'", 'kur', 'to', 's', 'is', "'", ':', '', '-', '0.3', '3', '27', '66',

"This scatterplot displays the clustering results of groups between 'woman' and 'votes' relationships. The data points for 'woman' typically have a mean of approximately 76.465 and a standard deviation of around 61.767, while the data points for 'votes' have a mean of roughly -4067.31 and a standard deviation of approximately 2982.875. There are a total of 10 groups visible in the chart, and there may be other clusters or patterns present that are difficult to interpret."

In [26]:
idx=3

import torch
outputs = model.generate(input_ids=torch.tensor(tokenized_dataset["test"][idx]['input_ids']).unsqueeze(0).to('cuda'),max_new_tokens=1024)
decoded_preds = tokenizer.batch_decode(tokenized_dataset["test"][idx]['input_ids'], skip_special_tokens=True)
print(decoded_preds)
decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(decoded_preds)
tokenized_dataset["test"][idx]['text']

['', '<unk>', "'", 'x', '-', 'axi', 's', '-', 'l', 'abel', "'", ':', '"', "'", '<unk>', "'", '"', ',', '', "'", 'y', '-', 'axi', 's', '-', 'l', 'abel', "'", ':', '"', '', "'", 'To', 'ront', 'o', "'", '",', '', "'", 'x', '-', 'axi', 's', '-', 'feature', 's', "'", ':', '[', '<unk>', "'", 'group', "'", ':', '', "'", 'dur', "'", ',', '', "'", 'color', "'", ':', '', "'", '#', '3', 'aff', 'b', 'c', "'", ',', '', "'", 'me', 'an', "'", ':', '', '89', '6.4', '26', '57', '84', '44', '56', '06', ',', '', "'", 's', 't', 'd', "'", ':', '5', '7.2', '224', '32', '146', '34', '43', '96', ',', '', "'", 'media', 'n', "'", ':', '', '89', '4.1', '20', '36', '330', '178', '42', ',', '', "'", 'min', "'", ':', '', '74', '1.6', '47', '360', '177', '81', '73', ',', '', "'", 'max', "'", ':', '', '106', '0.9', '10', '88', '450', '22', '66', ',', '', "'", 'ske', 'w', 'ness', "'", ':', '0.', '08', '184', '77', '15', '720', '43', '174', ',', '', "'", 'kur', 'to', 's', 'is', "'", ':', '', '-', '0.0', '21', '24', '79

'"This scatterplot shows the clustering results of groups between \'और\' and \'Toronto\' relationships. The data for \'और\' in this chart has a mean of 724.805 and a standard deviation of 1864.799, while the data for \'Toronto\' has a mean of 423.917 and a standard deviation of 300.293. There are a total of 7 groups, with colors assigned to each cluster based on their differences in data from the main cluster. The color names for these clusters are blue and red, while the group names for these clusters are \'rotor\' and \'missionaries\'."'

In [24]:
idx=4

import torch
outputs = model.generate(input_ids=torch.tensor(tokenized_dataset["test"][idx]['input_ids']).unsqueeze(0).to('cuda'),max_new_tokens=1024)
decoded_preds = tokenizer.batch_decode(tokenized_dataset["test"][idx]['input_ids'], skip_special_tokens=True)
print(decoded_preds)
decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(decoded_preds)
tokenized_dataset["test"][idx]['text']

['', '<unk>', "'", 'x', '-', 'axi', 's', '-', 'l', 'abel', "'", ':', '"', "'", 'bank', "'", '"', ',', '', "'", 'y', '-', 'axi', 's', '-', 'l', 'abel', "'", ':', '"', '', "'", 'CAT', "'", '",', '', "'", 'x', '-', 'axi', 's', '-', 'feature', 's', "'", ':', '[', '<unk>', "'", 'group', "'", ':', '', "'", 'K', 'a', "'", ',', '', "'", 'color', "'", ':', '', "'", '#', 'f', 'f', 'd', '500', "'", ',', '', "'", 'me', 'an', "'", ':', '', '-7', '89', '.', '69', '07', '55', '38', '80', '74', '4,', '', "'", 's', 't', 'd', "'", ':', '5.', '55', '42', '66', '70', '60', '15', '401', ',', '', "'", 'media', 'n', "'", ':', '', '-7', '89', '.', '2006', '09', '23', '48', '98', '5,', '', "'", 'min', "'", ':', '', '-8', '01.', '38', '68', '49', '72', '169', '45', ',', '', "'", 'max', "'", ':', '', '-', '777', '.', '74', '62', '12', '204', '72', '09', ',', '', "'", 'ske', 'w', 'ness', "'", ':', '', '-', '0.3', '60', '2', '20', '58', '18', '157', '18', '2,', '', "'", 'kur', 'to', 's', 'is', "'", ':', '', '-', '

"This scatterplot depicts the relationship between 'bank' and 'CAT', with a mean of -790.716 and a standard deviation of 5.584 for the 'bank' data, and a mean of 302.78 and a standard deviation of 1.773 for the 'CAT' data. There are a total of 9 groups, and the correlation coefficient is -0.047, indicating no linear relationship between the two datasets."

In [23]:
idx=5

import torch
outputs = model.generate(input_ids=torch.tensor(tokenized_dataset["test"][idx]['input_ids']).unsqueeze(0).to('cuda'),max_new_tokens=1024)
decoded_preds = tokenizer.batch_decode(tokenized_dataset["test"][idx]['input_ids'], skip_special_tokens=True)
print(decoded_preds)
decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(decoded_preds)
tokenized_dataset["test"][idx]['text']

['', '<unk>', "'", 'x', '-', 'axi', 's', '-', 'l', 'abel', "'", ':', '"', "'", 'after', 'n', 'o', 'on', "'", '"', ',', '', "'", 'y', '-', 'axi', 's', '-', 'l', 'abel', "'", ':', '"', '', "'", 'pen', 'insul', 'a', "'", '",', '', "'", 'x', '-', 'axi', 's', '-', 'feature', 's', "'", ':', '[', '<unk>', "'", 'group', "'", ':', '', "'", 'MD', "'", ',', '', "'", 'color', "'", ':', '', "'", '#', 'f', 'f', '4', '000', "'", ',', '', "'", 'me', 'an', "'", ':', '', '-', '80', '4.3', '81', '70', '72', '98', '62', '74', ',', '', "'", 's', 't', 'd', "'", ':', '', '63', '5.2', '58', '920', '375', '260', '3,', '', "'", 'media', 'n', "'", ':', '', '-8', '28', '.', '66', '81', '95', '41', '70', '70', '6,', '', "'", 'min', "'", ':', '', '-20', '66', '.', '158', '83', '168', '55', '16', ',', '', "'", 'max', "'", ':', '50', '6.6', '49', '68', '68', '17', '103', '2,', '', "'", 'ske', 'w', 'ness', "'", ':', '0.', '118', '510', '55', '2012', '86', '70', '2,', '', "'", 'kur', 'to', 's', 'is', "'", ':', '', '-',

"This scatterplot shows the clustering results of groups between 'afternoon' and 'peninsula' relationships. The data points for 'afternoon' typically have a mean of -398.212 and a standard deviation of 756.651, while the data points for 'peninsula' have a mean of -63.384 and a standard deviation of 54.584. A total of 7 distinct groups are visible in the chart. The colors used to represent other clusters that deviate from the main cluster are orange, blue, and green, while the group names for these clusters are 'MD', 'Kerry', and 'mutta'."

In [25]:
idx=6

import torch
outputs = model.generate(input_ids=torch.tensor(tokenized_dataset["test"][idx]['input_ids']).unsqueeze(0).to('cuda'),max_new_tokens=1024)
decoded_preds = tokenizer.batch_decode(tokenized_dataset["test"][idx]['input_ids'], skip_special_tokens=True)
print(decoded_preds)
decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(decoded_preds)
tokenized_dataset["test"][idx]['text']

['', '<unk>', "'", 'x', '-', 'axi', 's', '-', 'l', 'abel', "'", ':', '"', "'", 'r', 'egi', 'ment', "'", '"', ',', '', "'", 'y', '-', 'axi', 's', '-', 'l', 'abel', "'", ':', '"', '', "'", 'So', 'und', "'", '",', '', "'", 'x', '-', 'axi', 's', '-', 'feature', 's', "'", ':', '[', '<unk>', "'", 'group', "'", ':', '', "'", 'Vol', "'", ',', '', "'", 'color', "'", ':', '', "'", '#', '91', 'f', 'f', '66', "'", ',', '', "'", 'me', 'an', "'", ':', '', '-4', '76', '.', '52', '62', '58', '25', '132', '98', ',', '', "'", 's', 't', 'd', "'", ':', '10', '0.8', '2014', '255', '470', '66', '3,', '', "'", 'media', 'n', "'", ':', '', '-4', '87', '.', '65', '4', '34', '38', '27', '59', '16', ',', '', "'", 'min', "'", ':', '', '-6', '67', '.', '61', '77', '20', '90', '74', '91', ',', '', "'", 'max', "'", ':', '', '-21', '1.8', '19', '117', '49', '39', '62', '27', ',', '', "'", 'ske', 'w', 'ness', "'", ':', '0.', '65', '24', '74', '26', '64', '27', '137', '1,', '', "'", 'kur', 'to', 's', 'is', "'", ':', '',

'This scatterplot illustrates the connection between "regiment" and "Sound," with each point representing a group of data. The mean values for "regiment" are typically around -499.565, with a standard deviation of 128.553, while the mean values for "Sound" are around -872.002, with a standard deviation of 172.636. The correlation coefficient of this scatter plot is 0.658, indicating a strong linear relationship between the two data sets.'

In [27]:
idx=0

import torch
outputs = model.generate(input_ids=torch.tensor(tokenized_dataset["train"][idx]['input_ids']).unsqueeze(0).to('cuda'),max_new_tokens=1024)
decoded_preds = tokenizer.batch_decode(tokenized_dataset["train"][idx]['input_ids'], skip_special_tokens=True)
print(decoded_preds)
decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(decoded_preds)
tokenized_dataset["test"][idx]['text']

['', '<unk>', "'", 'x', '-', 'axi', 's', '-', 'l', 'abel', "'", ':', '"', "'", '<unk>', "'", '"', ',', '', "'", 'y', '-', 'axi', 's', '-', 'l', 'abel', "'", ':', '"', '', "'", 'sensor', "'", '",', '', "'", 'x', '-', 'axi', 's', '-', 'feature', 's', "'", ':', '[', '<unk>', "'", 'group', "'", ':', '', "'", 's', 'occ', 'er', "'", ',', '', "'", 'color', "'", ':', '', "'", '#', '00', '8', 'd', 'f', 'f', "'", ',', '', "'", 'me', 'an', "'", ':', '', '-12', '77', '.', '620', '73', '40', '80', '36', '21', ',', '', "'", 's', 't', 'd', "'", ':', '1', '0.0', '98', '80', '47', '29', '65', '43', '19', ',', '', "'", 'media', 'n', "'", ':', '', '-12', '77', '.', '28', '15', '98', '770', '33', '57', ',', '', "'", 'min', "'", ':', '', '-13', '06.', '30', '198', '38', '85', '74', '38', ',', '', "'", 'max', "'", ':', '', '-12', '46', '.', '184', '197', '52', '76', '25', ',', '', "'", 'ske', 'w', 'ness', "'", ':', '', '-', '0.0', '87', '66', '143', '36', '49', '73', '47', '2,', '', "'", 'kur', 'to', 's', '

"This scatterplot illustrates the relationship between 'bankruptcy' and 'fait,' with a mean of 1916.079 and a standard deviation of 13.227 for the 'bankruptcy' data, and a mean of -454.019 and a standard deviation of 171.864 for the 'fait' data. There are three groups in total, and the correlation coefficient is -0.01, indicating no linear relationship between the two variables."

In [28]:
idx=1

import torch
outputs = model.generate(input_ids=torch.tensor(tokenized_dataset["train"][idx]['input_ids']).unsqueeze(0).to('cuda'),max_new_tokens=1024)
decoded_preds = tokenizer.batch_decode(tokenized_dataset["train"][idx]['input_ids'], skip_special_tokens=True)
print(decoded_preds)
decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(decoded_preds)
tokenized_dataset["test"][idx]['text']

['', '<unk>', "'", 'x', '-', 'axi', 's', '-', 'l', 'abel', "'", ':', '"', "'", 'server', "'", '"', ',', '', "'", 'y', '-', 'axi', 's', '-', 'l', 'abel', "'", ':', '"', '', "'", 'ex', 'plan', 'ation', "'", '",', '', "'", 'x', '-', 'axi', 's', '-', 'feature', 's', "'", ':', '[', '<unk>', "'", 'group', "'", ':', '', "'", 'D', 'i', 'o', 's', "'", ',', '', "'", 'color', "'", ':', '', "'", '#', 'f', 'f', '4', '000', "'", ',', '', "'", 'me', 'an', "'", ':', '', '-3', '197', '.', '60', '84', '97', '25', '27', '19', ',', '', "'", 's', 't', 'd', "'", ':', '10', '2.1', '148', '95', '410', '94', '232', ',', '', "'", 'media', 'n', "'", ':', '', '-', '320', '4.5', '80', '37', '13', '74', '07', ',', '', "'", 'min', "'", ':', '', '-', '34', '32', '.', '169', '66', '57', '47', '47', ',', '', "'", 'max', "'", ':', '', '-2', '87', '7.0', '114', '37', '132', '28', '3,', '', "'", 'ske', 'w', 'ness', "'", ':', '', '0.4', '09', '40', '18', '23', '57', '260', '15', ',', '', "'", 'kur', 'to', 's', 'is', "'", '

'"This scatterplot displays the distribution of values over time for \'novice\' with a mean of -1245.172 and a standard deviation of 12.288. There are a total of 13 groups, and no other abnormalities are present."'